# String similarity: comparing Levenshtein distance and Tf-Idf Vectorizer methods

String similarity (Comparing 'hello world' and 'hello wolrd') is an important component of deduplication.    
This workbook makes uses of the flexible structure of the *Suricate* package to compare two methods:
- One using the classic Levenshtein distance
- The other one using a tokenization of the words, (either by character or word), using n-grams, and then using the cosine similarity

## 1. Prepare the environment

### 1.1. Libraries needed

In [28]:
# Standard libraries for data science + data visualization
import pandas as pd
import numpy as np

### 1.2. Load the two tables to be compared


In [18]:
n_lines = 100
from suricate.data.companies import getXst
df_X = getXst() # df_X is a list containing two datasets to compare [left, right]
left = df_X[0]
left.sample(5)

name  \
ix                                                                         
e67441ba-c61a-4e50-ae15-61401e231568                   ma componentes sl   
2f0901d9-1f1f-4efe-a974-05e8d6b7edea                    mike garwood ltd   
a9b7c40b-6448-407b-89f9-a7d0b040a6ea  selex sensors and airborne systems   
e649baee-018e-4473-b911-9f41d76e6818                      blumenhof frey   
83024ff7-2624-431f-b299-ff6fbdf0cebf                 battery direct gmbh   

                                                   street               city  \
ix                                                                             
e67441ba-c61a-4e50-ae15-61401e231568     pg el pino calle            seville   
2f0901d9-1f1f-4efe-a974-05e8d6b7edea        shelleys lane              alton   
a9b7c40b-6448-407b-89f9-a7d0b040a6ea   2 crewe road north          edinburgh   
e649baee-018e-4473-b911-9f41d76e6818  wa3rmbachstraerasse  unterschleissheim   
83024ff7-2624-431f-b299-ff6fbdf0cebf     1 ewald renz str      bad schonborn   

                                     postalcode         duns countrycode  
ix                                                                        
e67441ba-c61a-4e50-ae15-61401e231568      41016          NaN          ES  
2f0901d9-1f1f-4efe-a974-05e8d6b7edea       gu34          NaN          GB  
a9b7c40b-6448-407b-89f9-a7d0b040a6ea    eh5 2xs   23226769.0          GB  
e649baee-018e-4473-b911-9f41d76e6818      85716          NaN          DE  
83024ff7-2624-431f-b299-ff6fbdf0cebf      76669  331599808.0          DE

In [19]:
right = df_X[1]
right.sample(5)

name  \
ix                                                                  
83b28c2c-86f0-4d05-93c4-08e05b4a0b39           otis zi du cass ii   
46bed352-d9d4-4aef-bd97-83309fcc4c54               blumenhof frey   
43c007d9-4e4a-4e36-9714-2e06374da2e2        ceg alain vendrell ek   
ad10bc13-7e4d-4126-be1c-698a97c8b8db  honeywell aerospace vendome   
6ce89b84-d60a-42c6-8612-18f43319737e                   stahl gmbh   

                                                           street  \
ix                                                                  
83b28c2c-86f0-4d05-93c4-08e05b4a0b39                   saint jean   
46bed352-d9d4-4aef-bd97-83309fcc4c54   17a wa12rmbachstraayerasse   
43c007d9-4e4a-4e36-9714-2e06374da2e2                 max eyth str   
ad10bc13-7e4d-4126-be1c-698a97c8b8db  18 boulevard de l industrie   
6ce89b84-d60a-42c6-8612-18f43319737e            3 w maybach strae   

                                                   city postalcode  \
ix                                                                   
83b28c2c-86f0-4d05-93c4-08e05b4a0b39            l union      31240   
46bed352-d9d4-4aef-bd97-83309fcc4c54  unterschleissheim      85716   
43c007d9-4e4a-4e36-9714-2e06374da2e2     illerkirchberg      89171   
ad10bc13-7e4d-4126-be1c-698a97c8b8db              paris      75001   
6ce89b84-d60a-42c6-8612-18f43319737e         crailsheim      74564   

                                             duns countrycode  
ix                                                             
83b28c2c-86f0-4d05-93c4-08e05b4a0b39          NaN          FR  
46bed352-d9d4-4aef-bd97-83309fcc4c54          NaN          DE  
43c007d9-4e4a-4e36-9714-2e06374da2e2  342171451.0          DE  
ad10bc13-7e4d-4126-be1c-698a97c8b8db          NaN          FR  
6ce89b84-d60a-42c6-8612-18f43319737e  312901044.0          DE

In [20]:
n_possible_pairs= left.shape[0]*right.shape[0]
print('Two datasets of size {} and {}  rows yield:\n {} possible pairs to scan --> manually exhausting'.format(left.shape[0], right.shape[0], n_possible_pairs))

Two datasets of size 100 and 100  rows yield:
 10000 possible pairs to scan --> manually exhausting


## 2. How does the string comparator performs?

### 2.1. Using Levenshtein distance
The Levenshtein distance, and its derivative from the awesome *fuzzywuzzy* package can be used to compare each row of *left* against each row of *right*

In [33]:
from suricate.lrdftransformers import FuzzyConnector
t3 = FuzzyConnector(on='name', kind='simple')
simple_levenshtein = t3.fit_transform(X=df_X)

The results are more readable using the CartesianDataPasser from suricate

In [35]:
from suricate.lrdftransformers.cartesian import CartesianDataPasser
X_sbs = CartesianDataPasser().fit_transform(df_X).set_index(['ix_source', 'ix_target'])[['name_source', 'name_target']]
X_sbs['levenshteinsimplescore'] =  simple_levenshtein
X_sbs.sample(3)


name_source  \
ix_source                              ix_target                                                            
0a8de44c-737e-4f10-bb2c-2ae14687aa62 c2e5b14f-2d7e-44ec-b4fd-77251782fedd                  rdmadrid sl   
39e80128-8b75-44b9-a197-f253b2bb9fec 59de0cf8-cb34-4b30-a5cc-8ad39b44214b  alexander speith gmbh co kg   
2f0901d9-1f1f-4efe-a974-05e8d6b7edea 43c007d9-4e4a-4e36-9714-2e06374da2e2             mike garwood ltd   

                                                                                      name_target  \
ix_source                              ix_target                                                      
0a8de44c-737e-4f10-bb2c-2ae14687aa62 c2e5b14f-2d7e-44ec-b4fd-77251782fedd             marketline   
39e80128-8b75-44b9-a197-f253b2bb9fec 59de0cf8-cb34-4b30-a5cc-8ad39b44214b   compass group france   
2f0901d9-1f1f-4efe-a974-05e8d6b7edea 43c007d9-4e4a-4e36-9714-2e06374da2e2  ceg alain vendrell ek   

                                                                           levenshteinsimplescore  
ix_source                              ix_target                                                      
0a8de44c-737e-4f10-bb2c-2ae14687aa62 c2e5b14f-2d7e-44ec-b4fd-77251782fedd                    0.38  
39e80128-8b75-44b9-a197-f253b2bb9fec 59de0cf8-cb34-4b30-a5cc-8ad39b44214b                    0.26  
2f0901d9-1f1f-4efe-a974-05e8d6b7edea 43c007d9-4e4a-4e36-9714-2e06374da2e2                    0.27

Let's look at those "close matches" which are neither obvious mistakes nor identical strings

In [40]:
X_sbs.loc[(X_sbs['levenshteinsimplescore']>0.7) & (X_sbs['levenshteinsimplescore']<1.0)].sample(5)

name_source  \
ix_source                              ix_target                                                              
4c115719-4309-459e-8f48-8929761074f7 253ce464-33bd-41cc-a078-81f262216c45  hamilton sundstrand aerospace   
39082433-598a-49f1-b84b-6c3400305119 3b9f57b4-85af-47be-9530-c553a807f4a3          sinus electronic gmbh   
1b69ccb6-5901-4746-b1f9-8eef0d9a9870 85cc4387-2de3-4e43-97e1-a9d8d3c52fb6     nespresso deutschland gmbh   
be102af6-1552-480a-98fe-53eced051582 75b2984e-4929-4faf-a4fe-3025954327c9      aspen electronics limited   
a0a1b780-952c-4c09-978e-a34f8cc56eae ce8a3993-e248-48e8-a594-7718e3e530b3            pts automation gmbh   

                                                                                         name_target  \
ix_source                              ix_target                                                         
4c115719-4309-459e-8f48-8929761074f7 253ce464-33bd-41cc-a078-81f262216c45       hamilton sundstrand   
39082433-598a-49f1-b84b-6c3400305119 3b9f57b4-85af-47be-9530-c553a807f4a3          sinus electronic   
1b69ccb6-5901-4746-b1f9-8eef0d9a9870 85cc4387-2de3-4e43-97e1-a9d8d3c52fb6   oracle deutschland gmbh   
be102af6-1552-480a-98fe-53eced051582 75b2984e-4929-4faf-a4fe-3025954327c9  aspen electonics limited   
a0a1b780-952c-4c09-978e-a34f8cc56eae ce8a3993-e248-48e8-a594-7718e3e530b3       abb automation gmbh   

                                                                           levenshteinsimplescore  
ix_source                              ix_target                                                      
4c115719-4309-459e-8f48-8929761074f7 253ce464-33bd-41cc-a078-81f262216c45                    0.79  
39082433-598a-49f1-b84b-6c3400305119 3b9f57b4-85af-47be-9530-c553a807f4a3                    0.86  
1b69ccb6-5901-4746-b1f9-8eef0d9a9870 85cc4387-2de3-4e43-97e1-a9d8d3c52fb6                    0.78  
be102af6-1552-480a-98fe-53eced051582 75b2984e-4929-4faf-a4fe-3025954327c9                    0.98  
a0a1b780-952c-4c09-978e-a34f8cc56eae ce8a3993-e248-48e8-a594-7718e3e530b3                    0.84

#### The framework is fully compatible with open-source Scikit-Learn Machine Learning libraries

In [ ]:
t1 = VectorizerConnector(on='name', analyzer='word', ngram_range=(1,2))
t2 = VectorizerConnector(on='name', analyzer='char', ngram_range=(1,2))


In [ ]:
%%timeit
y1=t1.fit_transform(X=df_X)

In [ ]:
%%timeit
y2 = t2.fit_transform(X=df_X)

In [ ]:
%%timeit
y3 = t3.fit_transform(X=df_X)

# Make prediction using training data

In [ ]:
p1 = PipeLrClf(transformer=t1, classifier=Classifier())
y1_pred = pd.Series(
    data = p1.fit_predict(X=df_X, y=y_true),
    index=createmultiindex(X=df_X)
)
print(scores(y_true=y_true, y_pred=y1_pred))

In [ ]:
p2 = PipeLrClf(transformer=t2, classifier=Classifier())
y2_pred = pd.Series(
    data = p2.fit_predict(X=df_X, y=y_true),
    index=createmultiindex(X=df_X)
)
print(scores(y_true=y_true, y_pred=y2_pred))

In [ ]:
p3 = PipeLrClf(transformer=t3, classifier=Classifier())
y3_pred = pd.Series(
    data = p3.fit_predict(X=df_X, y=y_true),
    index=createmultiindex(X=df_X)
)
print(scores(y_true=y_true, y_pred=y3_pred))